In [ ]:
from transformers import pipeline
model = "siebert/sentiment-roberta-large-english"

analysis = pipeline(task="sentiment-analysis",model=model) 
print(analysis("Something ain't right"))

In [1]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
#https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest?text=Covid+cases+are+increasing+fast%21
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

analysis = pipeline(task="sentiment-analysis",model=model,tokenizer=tokenizer) 
a = analysis("When twitter spaces ?")

print(a[0]['label'] + ' ' + str(a[0]['score']))

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


neutral 0.9254334568977356


In [85]:
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd
import numpy as np

#https://stackoverflow.com/questions/66160583/removing-nearly-duplicate-observations-python
def dedupe_partially_vectorized(df, threshold=1):
    """
    - Iterate through each row starting from the last; examine all previous rows for duplicates.  
    - If found, it is appended to a list of duplicate indices.
    """
    # convert field data to integers
    enc = OrdinalEncoder()
    X = enc.fit_transform(df[["tweet","user_id"]].to_numpy())



    """
    - loop through each row, starting from last
    - for each `row`, calculate hamming distance to all previous rows
    - if any such distance is `threshold` or less, mark `idx` as duplicate
    - loop ends at 2nd row (1st is by definition not a duplicate)
    """
    dupe_idx = []          
    count = 0
    for j in range(len(X) - 1):

        idx = len(X) - j - 1
        row = X[idx]
        prev_rows = X[0:idx]
        dists = np.sum(row != prev_rows, axis=1)
        if min(dists) <= threshold:
            dupe_idx.append(idx)
            count+=1
        dupe_idx = sorted(dupe_idx)
    df_dupes = df.iloc[dupe_idx]
    df_deduped = df.drop(df_dupes.index,axis=0)

    print("Number of counts" + str(count))
    return df_deduped,df_dupes

In [160]:
import difflib

dupe_df = None
def dedupe(df,threshold=0.85):
    dupe = df.tweet.apply(lambda row: difflib.get_close_matches(row, list(df[df.tweet!=row].tweet), cutoff = threshold))    
    df_dupes = dupe[dupe.astype(bool)]
    df_deduped = df.drop(df_dupes.index,axis=0)
    return df_deduped,df_dupes

NameError: name 'df_dupes' is not defined

In [164]:
dupe_df

43     [1. how horny do you have to be to rt porn on ...
51     [1. how horny do you have to be to rt porn on ...
62     [NIKE Hello twitter welcome to rootjazz🌼 806  ...
76     [💝 💝how horny do you have to be to rt porn on ...
84     [1. how horny do you have to be to rt porn on ...
87     [💝 💝how horny do you have to be to rt porn on ...
100    [NIKE Hello twitter welcome to rootjazz🌼 886  ...
105    [1. how horny do you have to be to rt porn on ...
114    [Do you remember when you joined Twitter? I do...
142    [1. how horny do you have to be to rt porn on ...
158    [1. how horny do you have to be to rt porn on ...
163    [1. how horny do you have to be to rt porn on ...
175    [Do you remember when you joined Twitter? I do...
176    [1. how horny do you have to be to rt porn on ...
198    [1. how horny do you have to be to rt porn on ...
215    [💝 💝how horny do you have to be to rt porn on ...
234    [1. how horny do you have to be to rt porn on ...
279    [1. how horny do you hav

1599681264532873216 2022-12-05 16:25:08 +0800 <mafe_jorge> #República_Española en [ f ] captura  5D Republicat a 🐦 Twitter per  📷 J®️GediThor2022 ❤️❤️❤️❤️❤️❤️ 💛💛💛💛💛💛 💜💜💜💜💜💜 #SyR por #CoherenciaDemocrática #FelizSemanaRepublicana  https://t.co/vE5t6EGKdZ
1599681263241396225 2022-12-05 16:25:08 +0800 <br_rvw> Twitter始めました🌸 出稼ぎ紹介してくれるスカウトさん 紹介お願い致します🙇‍♀️
1599681262918463488 2022-12-05 16:25:08 +0800 <konbupis> なんでTwitterに投稿したら音ズレするの🫠
1599681262335119360 2022-12-05 16:25:08 +0800 <C_A_Guardiola> Tonight on Twitter: parts of speech front and center; the lies we tell ourselves about cats; and both Frank and Mark read your Twitter confessional and affirm that they have had enough of your shit.
1599681262003933186 2022-12-05 16:25:08 +0800 <AEON_JAPAN> @chiiichann22  ご応募いただきありがとうございます。 Twitterキャンペーンの結果は…残念😢 キャンペーンは12/25(日)23:59まで！  クリスマスでお買い物を楽しもう🎄  🔽詳細  https://t.co/lj5Xk9mYJh
1599681261919879169 2022-12-05 16:25:08 +0800 <Colambole> @JulienFort4bis @Roxas_Sui @Acermendax @RReichstadt @MrTea4

In [163]:
import twint
import nest_asyncio
nest_asyncio.apply()

# Configure
c = twint.Config()
c.Search = "Twitter"
c.Limit = 200
c.Lang = "en"
c.Pandas = True

# Run
twint.run.Search(c)
Tweets_df = twint.storage.panda.Tweets_df

#Sub selects english tweets
en_df = Tweets_df.query("language == 'en'")
print("Length of tweets in English: " + str(len(en_df)))


#en_df.drop_duplicates(subset=['tweet'])

#de_df,dupe_df = dedupe_partially_vectorized(en_df)
#Drop rows that are very similar to prevent spam that may affect results
de_df,dupe_df = dedupe(en_df)

#Only select columns that are used
df = de_df[["tweet", "nlikes","nretweets","nreplies"]]



#Calculate traction of tweet
df["traction"]= df["nlikes"] * 2 + df["nreplies"] * 3 + df["nretweets"] * 4

#Sorts the dataframe according to the most number of likes
sorted_df = df.sort_values(by=['traction'], ascending=False)
sorted_df

1599681576270696449 2022-12-05 16:26:23 +0800 <__Lm_h> 配信休みます。特に人がいっぱい来る訳でもない枠ですが一応お知らせ。 枠周りは行きます。Twitterではいつも通りに生きてます。休みは無期限です。 意外とすぐ再開するかもしれないし、遅くなるかもしれないし、もうしないかもしれない
1599681575985504257 2022-12-05 16:26:23 +0800 <soshow_eth> Six days after ChatGPT was released, the number of users has exceeded 1 million.  And he told me that if I invite both @cz_binance  and @SBF_FTX  to the twitter space, they won't fight.😂  What do you all use it for?💙  https://t.co/nPVMwN0npE
1599681575108608002 2022-12-05 16:26:22 +0800 <CikedDigital> How to Schedule Tweets on Twitter (7 Effective Tools &amp; Tactics)  https://t.co/f260aMbhiK
1599681571342409729 2022-12-05 16:26:21 +0800 <mr_uni_marke> 本日もお疲れ様でした！ Twitterを初めて6月も初めて半年、、 自分の目標だった『年内に2000名のフォロワー様と繋がる』ことが達成できました、、😭😭 フォロワー様には本当に感謝です！！ 今後とも宜しくお願い致します！  #企業公式相互フォロー  #企業公式がお疲れ様を言い合う  #フォロバ  https://t.co/9V1XfDY9T7
1599681570243223552 2022-12-05 16:26:21 +0800 <AEON_JAPAN> @a_purin_ao  ご応募いただきありがとうございます。 Twitterキャンペーンの結果は…残念😢 キャンペーンは12/25(日)23:59まで！  クリ

<ipython-input-163-21e8b758b635>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["traction"]= df["nlikes"] * 2 + df["nreplies"] * 3 + df["nretweets"] * 4


,tweet,nlikes,nretweets,nreplies,traction
240,Something weird happened to my twitter in Nove...,7,1,1,21
29,it makes perfect sense for scaramouche to be a...,2,1,2,14
337,Happy Festive Season Twitter! 🎄 Like this ...,5,0,1,13
95,Do you remember when you joined Twitter? I don...,2,0,1,7
378,@yhaw_pizzaro @TescoDarlinton Twitter,2,0,1,7
...,...,...,...,...,...
182,@justinamash How about the guy who had his cam...,0,0,0,0
187,@elonmusk @FoxNews But the Constitution will n...,0,0,0,0
191,@Vernon_Go @happeehour @pulpliveworld They did...,0,0,0,0
192,"I'm going to kill myself, yo @elonmusk am I al...",0,0,0,0


1599681401531822081 2022-12-05 16:25:41 +0800 <JMiSawBar> @zillagozilla @NickAdamsinUSA The mainstream media is not in any way a singular monolithic entity. News orgs compete for ratings and engagement, and "the dems got twitter to remove a politician's kid's leaked dick pics" doesn't have nearly the shock value you're hoping for.
1599681400852004864 2022-12-05 16:25:41 +0800 <lady__flash> @bone_in_pieces Except vagueing to the whole of Twitter about them I guess
1599681400101548033 2022-12-05 16:25:41 +0800 <I6JDtFvWbM5djA6> これ俺だけかもしらんけど電車でTwitterのタイムライン見ててエッチなの出てきて気まずい
1599681399585480704 2022-12-05 16:25:40 +0800 <FBN_help> Hello @ojeefosa247, thank you for contacting First contact and we empathize with you on the issue raised. Kindly follow us on our twitter handle @FBN_help. We'd need your account number and branch visited via direct message to enable us investigate. #YouFirst
1599681399581286400 2022-12-05 16:25:40 +0800 <AEON_JAPAN> @05__app  ご応募いただきありがとうございます。 Twitterキャンペーンの結果は

In [98]:
sus = dedupe(df)

1      [how horny do you have to be to rt porn on twi...
2      [I'm single, can I get a husband on Twitter?  ...
8      [how horny do you have to be to rt porn on twi...
13     [how horny do you have to be to rt porn on twi...
22     [how horny do you have to be to rt porn on twi...
                             ...                        
389    [how horny do you have to be to rt porn on twi...
393    [I'm single, can I get a husband on Twitter?  ...
403    [My Twitter Interaction Circle  Generate yours...
404    [how horny do you have to be to rt porn on twi...
405    [Do you remember when you joined Twitter? I do...
Name: tweet, Length: 79, dtype: object

In [154]:
def s_analysis(tweet):
    analysed = analysis(tweet)
    return analysed[0]['label'], analysed[0]['score']

In [158]:
sorted_df["sentiment"], sorted_df["score"] = zip(*sorted_df["tweet"].apply(s_analysis))
sorted_df

,tweet,nlikes,nretweets,nreplies,traction,sentiment,score
1064,Causa da morte: a #CCXP22 gritando “YOU ARE BR...,2492,243,27,6037,positive,0.935704
1429,#KeanuReeves has taken the stage at #CCXP22 fo...,513,68,11,1331,neutral,0.855919
957,Why People love this guy ❤️ #KeanuReeves #Joh...,492,59,11,1253,positive,0.959775
1380,NEW: Keanu Reeves is ready to do damage in thi...,436,88,7,1245,neutral,0.618263
1523,They’re giving out this new #JohnWick4 poster ...,468,63,7,1209,neutral,0.556143
...,...,...,...,...,...,...,...
732,"In a world full of Joy Behar's, I'm going to c...",0,0,0,0,positive,0.560643
730,MLK A PORRA DO KEANU REEVES,0,0,0,0,neutral,0.847812
715,@elonmusk Elon bites the holy Apple. Apparen...,0,0,0,0,neutral,0.662710
713,@KidNamedCy @Reddit wholesome keanu reeves bab...,0,0,0,0,neutral,0.447716


In [159]:
score_df = sorted_df[sorted_df['score'] > 0.5]
score_df['sentiment'].value_counts()

positive    345
neutral     308
negative     84
Name: sentiment, dtype: int64